In [1]:
import os
import scipy.io as spio
import numpy as np
import plotly.graph_objects as pgo
import plotly
plotly.io.renderers.default = "notebook"

In [2]:
path = "../outputs/2022-07-19/8-solitons-random-144-repulsive/"
sim_files = os.listdir(path)
snap_files = [f for f in sim_files if f.startswith("snap-Psi")]
snap = spio.loadmat(os.path.join(path, snap_files[-1]))
Psi = np.stack([snap["Psi"][0, i] for i in range(3)], axis=0)
simConfig = spio.loadmat(os.path.join(path, "simConfig.mat"))["simConfig"]

In [3]:
Lbox = float(simConfig["Lbox"][0, 0][0, 0])
N = int(simConfig["N"][0, 0][0, 0])
print(f"N={N}, Lbox={Lbox}")

N=144, Lbox=100.0


In [4]:
Rho = np.sum(np.absolute(Psi), axis = 0)

In [5]:
def downscale(A, factor, axis):
	indexer = [slice(None) for _i in range(len(A.shape))]
	O = None
	for i in range(factor):
		indexer[axis] = slice(i, None, factor)
		iter_indexer = tuple(indexer)
		if O is None:
			O = A[iter_indexer]
		else:
			O += A[iter_indexer]
	return O / factor

def downscale_3d(A, factor):
	A = downscale(A, factor, 2)
	A = downscale(A, factor, 1)
	A = downscale(A, factor, 0)
	return A

In [ ]:
scale_factor = 4
grid_axis = np.linspace(-Lbox / 2, Lbox / 2, num=N//scale_factor)
GX, GY, GZ = np.meshgrid(grid_axis, grid_axis, grid_axis)
Val = downscale_3d(Rho, scale_factor)
fig = pgo.Figure(data = pgo.Volume(
	x = GX.flatten(),
	y = GY.flatten(),
	z = GZ.flatten(),
	value = np.log1p(Val).flatten(),
	opacity = 0.05,
	surface_count = 96
))
fig.show()